In [1]:
print("hello")

hello


In [2]:
from IncrementalTrainingApproach.iCaRL import iCaRLmodel

In [3]:
from models import get_feature_extractor
feature_extractor = get_feature_extractor('cct7_h')
trainer = iCaRLmodel(10,feature_extractor,128,10,2000,10,1)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
print("Hello")
print(trainer.model.feature.tokenizer)

Hello
Tokenizer(
  (conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU()
      (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
  )
  (flattener): Flatten(start_dim=2, end_dim=3)
)


In [9]:
trainer.model.feature.tokenizer.requires_grad_(False)


Tokenizer(
  (conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU()
      (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
  )
  (flattener): Flatten(start_dim=2, end_dim=3)
)